In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prfs
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, fbeta_score
from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler

import io
import os
import uvicorn
import numpy as np
import nest_asyncio
from enum import Enum
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import StreamingResponse

import pickle
from pydantic import BaseModel

## Load Predict Functions

In [2]:
load_sample = {
    "step":1,
    "type":"PAYMENT",
    "amount":9839.64,
    "nameOrig":"C1231006815",
    "oldbalanceOrig":170136.0,
    "newbalanceOrig":160296.36,
    "nameDest":"M1979787155",
    "oldbalanceDest":0.0,
    "newbalanceDest":0.0
}

In [3]:
with open('model.pickle', 'rb') as f:
    model = pickle.loads(f.read())

In [4]:
def rule_predict(load):
    if load['nameDest'][0] == "M":
        return "Not Fraud"
    elif load['type'] not in ['CASH_OUT', 'TRANSFER']:
        return "Not Fraud"
    elif load['amount'] == 0:
        return "Fraud"
    else:
        return None
    
def succession_counter(load):
    # Load counter file
    with open('succession_counter.pickle', 'rb') as f:
        df_succession_counter = pickle.loads(f.read())
    
    # Add 1 successive counter in transaction, if exists
    if (load['step'], load['nameDest']) in df_succession_counter.index:
        df_succession_counter.loc[(load['step'], load['nameDest'])] += 1   
    else:
        df_succession_counter.loc[(load['step'], load['nameDest'])] = 1

    # Update File
    with open('succession_counter.pickle', 'wb') as f:
        pickle.dump(df_succession_counter, f)
        
    return df_succession_counter.loc[(load['step'], load['nameDest'])]
    
def predict_ml(load):
    difforig = load['newbalanceOrig'] - load['oldbalanceOrig']
    diffdest = load['newbalanceDest'] - load['oldbalanceDest']
    X = np.array([load['amount'],
                  load['oldbalanceOrig'],
                  load['oldbalanceDest'],
                  difforig,
                  diffdest,
                  succession_counter(load),
                  (difforig == -diffdest) * 1,
                  (difforig == -load['amount']) * 1,
                  (diffdest == load['amount']) * 1,
                  (load['type'] == "TRANSFER") * 1
                 ])
    return "Fraud" if model.predict([X])  else "Not Fraud"

def predict_api(load):
    rule_result = rule_predict(load)
    if rule_result:
        return rule_result
    else:
        return predict_ml(load)

In [5]:
predict_api(load_sample)

'Not Fraud'

## API

In [6]:
class transaction(BaseModel):
    step: int
    type: str
    amount: float
    nameOrig: str
    oldbalanceOrig: float
    newbalanceOrig: float
    nameDest: str
    oldbalanceDest: float
    newbalanceDest: float
        
app = FastAPI(title='Transaction Fraud Detection')

@app.get("/")
def home():
    return ""

@app.post("/is-fraud")
def prediction(request: transaction):
    return predict_api(dict(request))

In [7]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [29908]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58449 - "POST /is-fraud HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [29908]
